In [1]:
import sbol
import pandas as pd
import os

In [2]:
path_to_build_request = ''

In [3]:
SBH_USER = ''
SBH_PASSWORD = ''

In [4]:
sbh_stage = sbol.PartShop('https://hub-staging.sd2e.org')
sbh_stage.spoof("https://hub.sd2e.org")
sbh_stage.login(SBH_USER, SBH_PASSWORD)

In [6]:
parts_doc = sbol.Document()

build_excel = pd.ExcelFile(path_to_build_request)
build_request = pd.read_excel(build_excel, 'Design')
ontology_terms = pd.read_excel(build_excel, 'Ontology Terms', index_col=0)

collection_name = build_request.loc['Collection Name:'].iloc[:1][1].values[0]
parts_doc.name = collection_name
parts_doc.displayId = '_'.join(collection_name.split(' '))
collection_desc = build_request.loc['Design Description':].iloc[1:2].index[0]
parts_doc.description = collection_desc

sbol.setHomespace('https://hub-staging.sd2e.org/user/sd2e/{collection}'.format(
    collection=parts_doc.displayId))

parts_info_df = build_request.loc['Part Name':'Composite DNA Parts']
parts_info_df.columns = parts_info_df.iloc[0]
parts_info_df = parts_info_df.drop(parts_info_df.index[0])
parts_info_df = parts_info_df.loc[parts_info_df.index.dropna()]
parts_info_df = parts_info_df.drop(['Composite DNA Parts'])

for part_name, part_info in parts_info_df.iterrows():
    part_displayid = '_'.join(part_name.split(' '))
    part_cd = sbol.ComponentDefinition(part_displayid, sbol.BIOPAX_DNA)
    part_cd.name = part_name
    part_cd.description = part_info['Description (Optional)']
    
    if not pd.isnull(part_info['Role']):
        if part_info['Role'] in ontology_terms.index:
            role_uri = ontology_terms.loc[part_info['Role']].values[0]
            part_cd.roles = part_cd.roles + [role_uri]
    
    if not pd.isnull(part_info['Sequence']):
        part_seq = sbol.Sequence('{}_sequence'.format(part_cd.displayId), part_info['Sequence'])
        part_cd.sequence = part_seq

    parts_doc.addComponentDefinition(part_cd)

In [140]:
sbh_stage.submit(parts_doc, sbol.getHomespace(), 0)

<Response [200]>